# Predicción de precios de las viviendas de Boston

## XGBoost en SageMaker (Batch Transform - Deploy)

Como introducción al uso de la API Python de alto nivel de SageMaker, veremos un problema relativamente simple. A saber, usaremos el conjunto de datos de vivienda de Boston para predecir el valor medio de una casa en el área de Boston Mass.

## Bosquejo general
Por lo general, cuando use una instancia de notebook con SageMaker, procederá con los siguientes pasos. Por supuesto, no todos los pasos deberán realizarse con cada proyecto. Además, hay mucho margen de variación en muchos de los pasos, como verá a lo largo de estas lecciones.

* Descargue o recupere los datos.
* Procesar / preparar los datos.
* Suba los datos procesados a S3.
* Entrenar a un modelo elegido.
* Pruebe el modelo entrenado (generalmente usando un trabajo de transformación por lotes).
* Implemente el modelo entrenado.
* Use el modelo desplegado.
En este cuaderno solo cubriremos los pasos 1 a 5, ya que solo queremos tener una idea del uso de SageMaker. En portátiles posteriores, hablaremos sobre la implementación de un modelo entrenado con mucho más detalle.

## Paso 0: configuración del portátil
Comenzamos configurando todos los bits necesarios necesarios para ejecutar nuestro portátil. Para comenzar eso significa cargar todos los módulos de Python que necesitaremos.

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import os

Además de los módulos anteriores, necesitamos importar los diversos bits de SageMaker que utilizaremos.

* sagemaker.session(): Este es un objeto que representa la sesión de SageMaker en la que estamos operando actualmente. Este objeto contiene información útil a la que tendremos que acceder más adelante, como nuestra región.

* get_execution_role(): Este es un objeto que representa el rol de IAM que estamos asignados actualmente. Cuando construyamos y lancemos el trabajo de capacitación más adelante, tendremos que decirle qué rol de IAM debería tener. Dado que nuestro caso de uso es relativamente simple, simplemente asignaremos al trabajo de capacitación el rol que tenemos actualmente.

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.predictor import csv_serializer

In [ ]:
session = sagemaker.Session()

role = get_execution_role()

## Paso 1: descargando los datos
Afortunadamente, este conjunto de datos se puede recuperar con sklearn, por lo que este paso es relativamente sencillo.

In [7]:
boston = load_boston()

## Paso 2: preparar y dividir los datos
Dado que se trata de datos tabulares limpios, no necesitamos realizar ningún procesamiento. Sin embargo, necesitamos dividir las filas del conjunto de datos en conjuntos de entrenamiento, prueba y validación.

In [ ]:
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = pd.Series(boston.target, name='PRICE')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

## Paso 3: subir los archivos de datos a S3
Cuando se construye un trabajo de capacitación utilizando SageMaker, se ejecuta un contenedor que realiza la operación de capacitación. Este contenedor tiene acceso a los datos almacenados en S3. Esto significa que necesitamos cargar los datos que queremos usar para el entrenamiento en S3. Además, cuando realizamos un trabajo de transformación por lotes, SageMaker espera que los datos de entrada se almacenen en S3. Podemos usar la API de SageMaker para hacer esto y ocultar algunos de los detalles.

### Guardar los datos localmente
Primero necesitamos crear los archivos csv de prueba, capacitación y validación que luego cargaremos en S3.

In [13]:
data_dir = '../Data/boston'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [ ]:
X_test.to_csv(os.path.join(data_dir, 'test.csv'), header=False, index=False)

pd.concat([y_val, X_val], axis=1).to_csv(os.path.join(data_dir, 'validation.csv'), header=False, index=False)
pd.concat([y_train, X_train], axis=1).to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

## Subir a S3
Dado que actualmente estamos ejecutando dentro de una sesión de SageMaker, podemos usar el objeto que representa esta sesión para cargar nuestros datos en el depósito S3 'predeterminado'. Tenga en cuenta que es una buena práctica proporcionar un prefijo personalizado (esencialmente una carpeta S3) para asegurarse de que no interfiere accidentalmente con los datos cargados desde algún otro cuaderno o proyecto.

In [ ]:
prefix = 'boston-xgboost-hl'

test_location = session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)
val_location = session.upload_data(os.path.join(data_dir, 'validation.csv'), key_prefix=prefix)
train_location = session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

## Paso 4: entrena el modelo XGBoost
Ahora que tenemos los datos de entrenamiento y validación cargados en S3, podemos construir nuestro modelo XGBoost y entrenarlo. Haremos uso de la API de SageMaker de alto nivel para hacer esto, lo que hará que el código resultante sea un poco más fácil de leer a costa de cierta flexibilidad.

Para construir un estimador, el objeto que deseamos entrenar, necesitamos proporcionar la ubicación de un contenedor que contenga el código de entrenamiento. Como estamos utilizando un algoritmo incorporado, Amazon proporciona este contenedor. Sin embargo, el nombre completo del contenedor es un poco largo y depende de la región en la que estamos operando. Afortunadamente, SageMaker proporciona un método útil llamado **get_image_uri** que construye el nombre de la imagen para nosotros.

Para usar el método **get_image_uri**, debemos proporcionarle nuestra región actual, que se puede obtener del objeto de sesión, y el nombre del algoritmo que deseamos usar. En este cuaderno usaremos XGBoost, sin embargo, puede probar otro algoritmo si lo desea. La lista de algoritmos integrados se puede encontrar en la lista de Parámetros comunes.

In [ ]:
container = get_image_uri(session.boto_region_name, 'xgboost')

# Construcción del objeto estimator
xgb = sagemaker.estimator.Estimator(container, # El nombre de la imagen del contenedor de entrenamiento
                                    role, # Rol que usamos (IAM)
                                    train_instance_count=1, # Número de instancias para el entrenamiento
                                    train_instance_type='ml.m4.xlarge', # Tipo de instancias para el entremaniento
                                    output_path='s3://{}/{}/output'.format(session.default_bucket(), prefix), # Lugar donde se guarda la salida
                                    sagemaker_session=session) # Sesión de SageMaker

Antes de pedirle a SageMaker que comience el trabajo de capacitación, probablemente deberíamos configurar cualquier hiperparámetro específico del modelo. Hay bastantes que se pueden configurar al usar el algoritmo XGBoost, a continuación se detallan algunos de ellos. Si desea cambiar los hiperparámetros a continuación o modificar otros adicionales, puede encontrar información adicional en la página del hiperparámetro XGBoost.

In [ ]:
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        objective='reg:linear',
                        early_stopping_rounds=10,
                        num_round=200)

Ahora que tenemos nuestro objeto estimador completamente configurado, es hora de entrenarlo. Para hacer esto, nos aseguramos de que SageMaker sepa que nuestros datos de entrada están en formato csv y luego ejecutamos el método de ajuste.

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data=val_location, content_type='csv')

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

# Segunda parte (Despliegue del modelo)

## Paso 5: prueba el modelo entrenado
Saltaremos este paso por ahora. Seguiremos probando nuestro modelo entrenado, pero lo haremos utilizando el modelo implementado, en lugar de configurar un trabajo de transformación por lotes.

## Paso 6: Implemente el modelo entrenado
Ahora que hemos ajustado nuestro modelo a los datos de entrenamiento, utilizando los datos de validación para evitar el sobreajuste, podemos implementar nuestro modelo y probarlo. La implementación es muy simple cuando usamos la API de alto nivel, solo necesitamos llamar al método de **deploy** de nuestro estimador capacitado.

**NOTA**: Al implementar un modelo, le está pidiendo a SageMaker que inicie una instancia de proceso que esperará a que se le envíen los datos. Como resultado, esta instancia de proceso continuará ejecutándose hasta que la cierre. Es importante saber esto, ya que el costo de un punto final implementado depende de cuánto tiempo ha estado funcionando.

En otras palabras, **si ya no está utilizando un punto final desplegado, ¡apáguelo!**

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

## Paso 7: usa el modelo
Ahora que nuestro modelo está entrenado y desplegado, podemos enviarle los datos de prueba y evaluar los resultados. Aquí, debido a que nuestros datos de prueba son tan pequeños, podemos enviarlos todos utilizando una sola llamada a nuestro punto final. Si nuestro conjunto de datos de prueba fuera más grande, tendríamos que dividirlo y enviar los datos en fragmentos, asegurándonos de acumular los resultados.

In [ ]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer

y_pred = xgb_predictor.predict(X_test.values).decode('utf-8')
y_pred = np.fromstring(y_pred, sep=',')

Para ver qué tan bien funciona nuestro modelo, podemos crear un diagrama de dispersión simple entre los valores predichos y los reales. Si el modelo fuera completamente exacto, el diagrama de dispersión resultante se vería como la línea $x = y$. Como podemos ver, nuestro modelo parece haber funcionado bien, pero hay margen de mejora.

In [ ]:
plt.scatter(y_test, y_pred)
plt.xlabel('Median Price')
plt.ylabel('Predicted Price')
plt.title('Median Price vs Predicted Price')

## Eliminar el Endpoint
Como ya no usamos el modelo implementado, debemos asegurarnos de cerrarlo. Recuerde que debe pagar por el período de tiempo que se despliega su punto final, de modo que cuanto más tiempo se deja en funcionamiento, más cuesta.

In [ ]:
xgb_predictor.delete_endpoint()

## Opcional: limpiar
La instancia de notebook predeterminada en SageMaker no tiene mucho espacio en disco disponible. A medida que continúe completando y ejecutando cuadernos, eventualmente llenará este espacio en disco, lo que generará errores que pueden ser difíciles de diagnosticar. Una vez que haya terminado de usar un cuaderno, es una buena idea eliminar los archivos que creó en el camino. Por supuesto, puede hacerlo desde la terminal o desde el hub del portátil si lo desea. La celda a continuación contiene algunos comandos para limpiar los archivos creados desde el cuaderno.

In [ ]:
!rm $data_dir/*

!rmdir $data_dir